pip install notebook  
pip install langchain  
ollama pull llama3.1     
pip install -qU langchain-ollama
pip install -qU langchain_community pypdf


# Пример использования с доки

In [1]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.1",
    temperature=0,
)

In [2]:
from langchain_core.messages import AIMessage

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='The translation of "I love programming" from English to French is:\n\n"J\'adore programmer."', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2024-11-30T10:34:56.7832386Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5971484200, 'load_duration': 4977125800, 'prompt_eval_count': 35, 'prompt_eval_duration': 217000000, 'eval_count': 22, 'eval_duration': 432000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-f3bcc130-baf4-4008-9326-51f58c1322d9-0', usage_metadata={'input_tokens': 35, 'output_tokens': 22, 'total_tokens': 57})

In [3]:
print(ai_msg.content)

The translation of "I love programming" from English to French is:

"J'adore programmer."


In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

AIMessage(content='Ich liebe Programmieren! (I love programming!)', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2024-11-30T10:35:48.6727252Z', 'done': True, 'done_reason': 'stop', 'total_duration': 254650200, 'load_duration': 15178200, 'prompt_eval_count': 30, 'prompt_eval_duration': 9000000, 'eval_count': 11, 'eval_duration': 229000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-6a12ebce-82ad-488e-9e6a-a16a16f709e5-0', usage_metadata={'input_tokens': 30, 'output_tokens': 11, 'total_tokens': 41})

In [5]:
from typing import List

from langchain_core.tools import tool
from langchain_ollama import ChatOllama


@tool
def validate_user(user_id: int, addresses: List[str]) -> bool:
    """Validate user using historical addresses.

    Args:
        user_id (int): the user ID.
        addresses (List[str]): Previous addresses as a list of strings.
    """
    return True


llm = ChatOllama(
    model="llama3.1",
    temperature=0,
).bind_tools([validate_user])

result = llm.invoke(
    "Could you validate user 123? They previously lived at "
    "123 Fake St in Boston MA and 234 Pretend Boulevard in "
    "Houston TX."
)
result.tool_calls

[{'name': 'validate_user',
  'args': {'addresses': '["123 Fake St, Boston MA", "234 Pretend Boulevard, Houston TX"]',
   'user_id': 123},
  'id': '8ec309ce-b74a-4464-9518-8a94cf44f243',
  'type': 'tool_call'}]

# Загрузка pdf

In [ ]:
# pip install -qU langchain_community pypdf

In [7]:
from langchain_community.document_loaders import PyPDFLoader

loader_1 = PyPDFLoader(
    "../data/sql_primer.pdf",
)

loader_2 = PyPDFLoader(
    "../data/Учебное_пособие_#Работа_с_СУБД_PostgreSQL#_4 (2).pdf",
)

In [14]:
docs = loader_1.load()

In [15]:
# кол-во стр в 1 файле
print(len(docs))

337


In [9]:
# вывод страницы 0
docs[0]

Document(metadata={'source': '../data/sql_primer.pdf', 'page': 0}, page_content='')

In [11]:
docs[2]

Document(metadata={'source': '../data/sql_primer.pdf', 'page': 2}, page_content='УДК 004.655\nББК 32.973.26-018.2\nМ79\nМоргунов, Е. П.\nМ79 PostgreSQL. Основы языка SQL: учеб. пособие / Е. П. Моргунов; под ред.\nЕ. В. Рогова, П. В. Лузанова. — СПб.: БХВ-Петербург, 2018. — 336 с.: ил.\nISBN 978-5-9775-4022-3\nУчебно-практическое пособие охватывает первую, базовую, часть учеб-\nного курса по языку SQL, созданного при участии российской компании\nPostgres Professional. Учебный материал излагается в расчете на использо-\nвание системы управления базами данных PostgreSQL. Рассмотрено созда-\nние рабочей среды, описаны языки определения данных и основные опе-\nрации выборки и изменения данных. Показаны примеры использования\nтранзакций, уделено внимание методам оптимизации запросов. Матери-\nал сопровождается многочисленными практическими примерами. Посо-\nбие может использоваться как для самостоятельного обучения, так и при\nпроведении занятий под руководством преподавателя.\nДля программи

In [16]:
docs += loader_2.load()

In [17]:
# кол-во стр в обоих файлах
print(len(docs))

512


In [21]:
print(docs[0].metadata)
print(docs[-1].metadata)

{'source': '../data/sql_primer.pdf', 'page': 0}
{'source': '../data/Учебное_пособие_#Работа_с_СУБД_PostgreSQL#_4 (2).pdf', 'page': 174}


# Embeddings

In [22]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3.1")

In [23]:
embeddings.embed_query("Hello, world!")

[-0.0035360663,
 -0.020200199,
 0.0030448185,
 0.010780741,
 -0.014586409,
 0.0015043701,
 0.0012104829,
 0.026147638,
 0.017949618,
 0.009873056,
 -0.008495268,
 0.008828553,
 -0.0001375843,
 -0.0048659546,
 0.008771579,
 0.0099633755,
 -0.016118934,
 0.0070558046,
 -0.013593657,
 0.0019665316,
 -0.008797875,
 -0.015740685,
 0.009450066,
 0.0202508,
 -0.012679806,
 -0.0034913633,
 0.0001920688,
 -0.03543205,
 0.0036499936,
 -0.011499475,
 0.0032963906,
 0.0063709333,
 -0.013327344,
 0.0069170147,
 0.060678035,
 -0.0021593317,
 -0.00044474576,
 -0.004472679,
 0.004464258,
 0.0078990515,
 0.0024256313,
 0.0034928326,
 0.0034406113,
 -0.004503121,
 0.016225282,
 -0.0028353757,
 0.012435014,
 -0.0012807585,
 0.027133444,
 -0.000846316,
 0.0028284967,
 -0.0011409324,
 -0.0035738256,
 0.0058319685,
 -0.0030761752,
 -0.00014985166,
 0.008753172,
 0.006115109,
 -0.015203832,
 0.011434536,
 -0.021077963,
 0.012027711,
 -0.009974325,
 0.00039568142,
 -0.008573641,
 0.0049076867,
 -0.023565575,


In [26]:
# input_texts = ["Документ 1...", "Документ 2..."]
# можно не объединять файлы
docs_page_content = "\n\n".join([d.page_content for d in docs])

input_texts = [docs_page_content]
vectors = embeddings.embed_documents(input_texts)
print(len(vectors))
# Первые 3 координаты для первого вектора
print(vectors[0][:3])

1
[-0.018857528, -0.028516801, -0.023277575]


In [30]:
print(len(vectors[0]))

4096


# Retrievers

# https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/

Ретривер — это интерфейс, который возвращает документы по неструктурированному запросу. Он более универсален, чем векторное хранилище. Ретривер не должен хранить документы, он должен только возвращать (или извлекать) их. Векторные хранилища могут использоваться в качестве основы для ретривера, но существуют и другие типы ретриверов.

Ретриверы принимают строку query в качестве входных данных и возвращают список Document в качестве выходных данных.

In [31]:
# надо решить протестить различные типы и размеры чанков
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("What did the president say about technology?")

ModuleNotFoundError: No module named 'langchain_openai'

# Использование согласно заданию

задание предполагает реализацию RAG в базовом варианте (загрузка документов, разделение их на чанки, векторизация чанков, размещение чанков в векторном хранилище, организация цепочки для извлечения релевантных чанков и получения ответа на вопрос), использование продвинутых техник RAG (MMR, Reranking, Ensemble retriever и др.) не требуется

## Загрузка

In [32]:
from langchain_community.document_loaders import PyPDFLoader

loader_1 = PyPDFLoader(
    "../data/sql_primer.pdf",
)

loader_2 = PyPDFLoader(
    "../data/Учебное_пособие_#Работа_с_СУБД_PostgreSQL#_4 (2).pdf",
)

In [33]:
docs = loader_1.load()

In [34]:
docs += loader_2.load()

## Разделение на чанки

https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/
чанки и хранилище векторов

https://python.langchain.com/docs/how_to/multi_vector/#smaller-chunks чета продвинутое

# Прочее
- параметры для модели https://python.langchain.com/api_reference/ollama/chat_models/langchain_ollama.chat_models.ChatOllama.html
- репа модели https://github.com/ollama/ollama
- скачать модель https://ollama.com/library/llama3.1 (скачать файл, установить и запустить приложение, а после скачать модель(ollama pull llama3.1). Взаимодействие есть в начале файла)
- что такое RAG https://habr.com/ru/articles/779526/